In [ ]:
import gradio as gr
def greet(name, intensity):
  return "Hello, " + name + "!" * int(intensity)
demo = gr.Interface(
  fn=greet,
  inputs=["text", "slider"],
  outputs=["text"],
)
demo.launch(server_name="127.0.0.1", server_port= 7860)

## Image captioning using pretrained Hugging face model

In [ ]:
# import gradio as gr
# from transformers import BlipProcessor, BlipForConditionalGeneration
# from PIL import Image

# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# def generate_caption(image):
#     # Now directly using the PIL Image object
#     inputs = processor(images=image, return_tensors="pt")
#     outputs = model.generate(**inputs)
#     caption = processor.decode(outputs[0], skip_special_tokens=False)
#     return caption

# def caption_image(image):
#     """
#     Takes a PIL Image input and returns a caption.
#     """
#     try:
#         caption = generate_caption(image)
#         return caption
#     except Exception as e:
#         return f"An error occurred: {str(e)}"

# iface = gr.Interface(
#     fn=caption_image,
#     inputs=gr.Image(type="pil"),
#     outputs="text",
#     title="Image Captioning with BLIP",
#     description="Upload an image to generate a caption."
# )

# iface.launch(server_name="127.0.0.1", server_port= 7860)

## Image Classification in Pytorch

In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True).eval()

Using cache found in /home/codespace/.cache/torch/hub/pytorch_vision_v0.6.0


In [2]:
import requests
from PIL import Image
from torchvision import transforms

In [3]:
#download human-readable lables for Imagenet
response = requests.get("https://git.io/JJkYN")
lables = response.text.split("\n")


In [8]:
def predict(inp):
    inp = transforms.ToTensor()(inp).unsqueeze(0)
    with torch.no_grad():
        prediction = torch.nn.functional.softmax(model(inp)[0],dim=0)
        confidence = {lables[i]:float(prediction[i]) for i in range(1000)}
    return confidence
        
        

In [ ]:
import gradio as gr

gr.Interface(
    fn = predict,
    inputs = gr.Image(type='pil'),
    outputs = gr.Label(num_top_classes=3),
    examples = ["/content/lion.jpg", "/content/cheetah.jpg"]
).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Simple LLM using Gradio interface

In [ ]:
# Import the necessary packages
# from ibm_watsonx_ai.foundation_models import ModelInference
# from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
# from ibm_watsonx_ai import Credentials
# from langchain_ibm import WatsonxLLM

import gradio as gr
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# Specify the model and project settings 
# (make sure the model you wish to use is commented out, and other models are commented)
#model_id = 'mistralai/mixtral-8x7b-instruct-v01' # Specify the Mixtral 8x7B model
model_id = 'llama-3.1-8b-instant' #'ibm/granite-3-3-8b-instruct' # Specify IBM's Granite 3.3 8B model
gpt_model_id ='openai/gpt-oss-120b'
# Set the necessary parameters
# parameters = {
#     GenParams.MAX_NEW_TOKENS: 256,  # Specify the max tokens you want to generate
#     GenParams.TEMPERATURE: 0.3, # This randomness or creativity of the model's responses
# }

project_id = "skills-network"

# Wrap up the model into WatsonxLLM inference
llm = ChatGroq(model=model_id, temperature=0.3)
# watsonx_llm = WatsonxLLM(
#     model_id=model_id,
#     url="https://us-south.ml.cloud.ibm.com",
#     project_id=project_id,
#     params=parameters,
# )

# Get the query from the user input
# query = input("Please enter your query: ")

# # Print the generated response
# print(watsonx_llm.invoke(query))

def generate_response(prompt_txt):
    generated_response = llm.invoke(prompt_txt)
    return generated_response.content

#Create Gradio interface
chat_completion = gr.Interface(
    fn = generate_response,
    allow_flagging="never",
    inputs=gr.Textbox(label="Input",lines=2,placeholder="Type your question here..."),
    outputs = gr.Textbox(label="Output"),
    title = "Watsonx ai chatbot",
    description="Ask any question and the chatbot will try to answer"
)

chat_completion.launch(server_name="127.0.0.1", server_port=7860, share=True)


/workspaces/RAG-AgenticAI/venv/lib/python3.12/site-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://ef2317c20eae86a9f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [21]:
llm.invoke("what is ML in 20 words?").content

'Machine Learning (ML) is a subset of AI that enables systems to learn from data without explicit programming rules.'

In [22]:
gpt_model_id = 'openai/gpt-oss-120b'
gpt_llm = ChatGroq(model=gpt_model_id, temperature=0.3)

In [23]:
gpt_llm.invoke("what is ML in 20 words?").content

'Machine Learning enables computers to learn patterns from data, improving performance on tasks without explicit programming and adapt over time.'